In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 3
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC10(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC10(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC10(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.06 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        
        #------------------------------------------------
        #需要調整的地方
        #------------------------------------------------
        #--要用幾秒訓練 data_1S/data_3S/data_1S
        #--學習率
        #--epoch
        if(TL==0):
            trainData = data_1S
            epoch = 90
            lr = 0.01
        elif(TL==1):
            trainData = data_3S
            epoch = 90
            lr = 0.01
        else:
            trainData = data_5S
            epoch = 90
            lr = 0.01
        #------------------------------------------------
    

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型
        model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
        #測試準確度
        test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
        test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
        test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/90 Train_Loss: 1.2818806171417236 Train_acc: 35.937500%
Train Epoch: 2/90 Train_Loss: 0.8912385702133179 Train_acc: 63.562500%
Train Epoch: 3/90 Train_Loss: 0.7570628523826599 Train_acc: 75.250000%
Train Epoch: 4/90 Train_Loss: 0.34129032492637634 Train_acc: 82.250000%
Train Epoch: 5/90 Train_Loss: 0.37529006600379944 Train_acc: 85.125000%
Train Epoch: 6/90 Train_Loss: 0.32357943058013916 Train_acc: 88.125000%
Train Epoch: 7/90 Train_Loss: 0.2393944412469864 Train_acc: 89.875000%
Train Epoch: 8/90 Train_Loss: 0.24268445372581482 Train_acc: 91.625000%
Train Epoch: 9/90 Train_Loss: 0.18074460327625275 Train_acc: 91.500000%
Train Epoch: 10/90 Train_Loss: 0.15473318099975586 Train_acc: 93.187500%
Train Epoch: 11/90 Train_Lo

Train Epoch: 15/90 Train_Loss: 0.06716081500053406 Train_acc: 98.958336%
Train Epoch: 16/90 Train_Loss: 0.01914886385202408 Train_acc: 98.854172%
Train Epoch: 17/90 Train_Loss: 0.07197058945894241 Train_acc: 98.958336%
Train Epoch: 18/90 Train_Loss: 0.008794580586254597 Train_acc: 98.958336%
Train Epoch: 19/90 Train_Loss: 0.009317232295870781 Train_acc: 98.958336%
Train Epoch: 20/90 Train_Loss: 0.02029789797961712 Train_acc: 98.958336%
Train Epoch: 21/90 Train_Loss: 0.021515216678380966 Train_acc: 98.958336%
Train Epoch: 22/90 Train_Loss: 0.0667816549539566 Train_acc: 98.958336%
Train Epoch: 23/90 Train_Loss: 0.0615437887609005 Train_acc: 98.854172%
Train Epoch: 24/90 Train_Loss: 0.010892722755670547 Train_acc: 98.958336%
Train Epoch: 25/90 Train_Loss: 0.020597755908966064 Train_acc: 98.958336%
Train Epoch: 26/90 Train_Loss: 0.017062952741980553 Train_acc: 98.958336%
Train Epoch: 27/90 Train_Loss: 0.0006928658112883568 Train_acc: 98.958336%
Train Epoch: 28/90 Train_Loss: 0.009068366140

Train Epoch: 31/90 Train_Loss: 0.0004838037712033838 Train_acc: 100.000000%
Train Epoch: 32/90 Train_Loss: 0.00039051545900292695 Train_acc: 100.000000%
Train Epoch: 33/90 Train_Loss: 0.0006201728247106075 Train_acc: 100.000000%
Train Epoch: 34/90 Train_Loss: 0.0007256354438140988 Train_acc: 100.000000%
Train Epoch: 35/90 Train_Loss: 0.0009909850778058171 Train_acc: 100.000000%
Train Epoch: 36/90 Train_Loss: 0.0003604679077398032 Train_acc: 100.000000%
Train Epoch: 37/90 Train_Loss: 0.0004991202731616795 Train_acc: 100.000000%
Train Epoch: 38/90 Train_Loss: 0.0011316726449877024 Train_acc: 100.000000%
Train Epoch: 39/90 Train_Loss: 0.0008631320670247078 Train_acc: 100.000000%
Train Epoch: 40/90 Train_Loss: 0.0004865339433308691 Train_acc: 100.000000%
Train Epoch: 41/90 Train_Loss: 0.0010083779925480485 Train_acc: 100.000000%
Train Epoch: 42/90 Train_Loss: 0.0005216591525822878 Train_acc: 100.000000%
Train Epoch: 43/90 Train_Loss: 0.00034604885149747133 Train_acc: 100.000000%
Train Epoc

Train Epoch: 46/90 Train_Loss: 0.2607947885990143 Train_acc: 93.562500%
Train Epoch: 47/90 Train_Loss: 0.08578889816999435 Train_acc: 94.187500%
Train Epoch: 48/90 Train_Loss: 0.11157878488302231 Train_acc: 94.437500%
Train Epoch: 49/90 Train_Loss: 0.18740472197532654 Train_acc: 94.437500%
Train Epoch: 50/90 Train_Loss: 0.07824159413576126 Train_acc: 94.625000%
Train Epoch: 51/90 Train_Loss: 0.1866835057735443 Train_acc: 94.312500%
Train Epoch: 52/90 Train_Loss: 0.08033963292837143 Train_acc: 93.937500%
Train Epoch: 53/90 Train_Loss: 0.12894630432128906 Train_acc: 94.687500%
Train Epoch: 54/90 Train_Loss: 0.17823386192321777 Train_acc: 94.750000%
Train Epoch: 55/90 Train_Loss: 0.10890170186758041 Train_acc: 94.375000%
Train Epoch: 56/90 Train_Loss: 0.09030814468860626 Train_acc: 94.125000%
Train Epoch: 57/90 Train_Loss: 0.1687266081571579 Train_acc: 94.562500%
Train Epoch: 58/90 Train_Loss: 0.09936533868312836 Train_acc: 94.687500%
Train Epoch: 59/90 Train_Loss: 0.048309460282325745 Tr

Train Epoch: 63/90 Train_Loss: 0.01514635980129242 Train_acc: 98.854172%
Train Epoch: 64/90 Train_Loss: 0.0005805381806567311 Train_acc: 98.958336%
Train Epoch: 65/90 Train_Loss: 0.03407188504934311 Train_acc: 98.854172%
Train Epoch: 66/90 Train_Loss: 0.03691122308373451 Train_acc: 98.854172%
Train Epoch: 67/90 Train_Loss: 0.0008896643994376063 Train_acc: 98.854172%
Train Epoch: 68/90 Train_Loss: 0.022833840921521187 Train_acc: 98.854172%
Train Epoch: 69/90 Train_Loss: 0.0007414494757540524 Train_acc: 98.854172%
Train Epoch: 70/90 Train_Loss: 0.029803859069943428 Train_acc: 98.854172%
Train Epoch: 71/90 Train_Loss: 0.021214690059423447 Train_acc: 98.854172%
Train Epoch: 72/90 Train_Loss: 0.022144628688693047 Train_acc: 98.854172%
Train Epoch: 73/90 Train_Loss: 0.0202710572630167 Train_acc: 98.854172%
Train Epoch: 74/90 Train_Loss: 0.00042247187229804695 Train_acc: 98.854172%
Train Epoch: 75/90 Train_Loss: 0.02153407223522663 Train_acc: 98.854172%
Train Epoch: 76/90 Train_Loss: 0.044366

...

Train Epoch: 31/90 Train_Loss: 0.0992172360420227 Train_acc: 94.562500%
Train Epoch: 32/90 Train_Loss: 0.10348311066627502 Train_acc: 94.812500%
Train Epoch: 33/90 Train_Loss: 0.10400624573230743 Train_acc: 94.375000%
Train Epoch: 34/90 Train_Loss: 0.15089572966098785 Train_acc: 94.687500%
Train Epoch: 35/90 Train_Loss: 0.17113159596920013 Train_acc: 94.375000%
Train Epoch: 36/90 Train_Loss: 0.18884946405887604 Train_acc: 94.312500%
Train Epoch: 37/90 Train_Loss: 0.08860953152179718 Train_acc: 94.437500%
Train Epoch: 38/90 Train_Loss: 0.060219861567020416 Train_acc: 94.750000%
Train Epoch: 39/90 Train_Loss: 0.08965110033750534 Train_acc: 94.812500%
Train Epoch: 40/90 Train_Loss: 0.19975991547107697 Train_acc: 94.312500%
Train Epoch: 41/90 Train_Loss: 0.08426423370838165 Train_acc: 94.812500%
Train Epoch: 42/90 Train_Loss: 0.08158707618713379 Train_acc: 94.812500%
Train Epoch: 43/90 Train_Loss: 0.15542617440223694 Train_acc: 94.562500%
Train Epoch: 44/90 Train_Loss: 0.2179838865995407 T

Train Epoch: 48/90 Train_Loss: 0.03195135295391083 Train_acc: 98.750008%
Train Epoch: 49/90 Train_Loss: 0.0135369086638093 Train_acc: 98.854172%
Train Epoch: 50/90 Train_Loss: 0.05090941861271858 Train_acc: 98.854172%
Train Epoch: 51/90 Train_Loss: 0.035302601754665375 Train_acc: 98.854172%
Train Epoch: 52/90 Train_Loss: 0.002005738904699683 Train_acc: 98.854172%
Train Epoch: 53/90 Train_Loss: 0.04649000242352486 Train_acc: 98.854172%
Train Epoch: 54/90 Train_Loss: 0.02125646360218525 Train_acc: 98.854172%
Train Epoch: 55/90 Train_Loss: 0.08459202200174332 Train_acc: 98.854172%
Train Epoch: 56/90 Train_Loss: 0.053080856800079346 Train_acc: 98.854172%
Train Epoch: 57/90 Train_Loss: 0.0006767017184756696 Train_acc: 98.854172%
Train Epoch: 58/90 Train_Loss: 0.03223415091633797 Train_acc: 98.854172%
Train Epoch: 59/90 Train_Loss: 0.019441725686192513 Train_acc: 98.854172%
Train Epoch: 60/90 Train_Loss: 0.020944921299815178 Train_acc: 98.854172%
Train Epoch: 61/90 Train_Loss: 0.010219762101

Train Epoch: 63/90 Train_Loss: 0.0004790720413438976 Train_acc: 100.000000%
Train Epoch: 64/90 Train_Loss: 0.0002558569540269673 Train_acc: 100.000000%
Train Epoch: 65/90 Train_Loss: 0.001597785740159452 Train_acc: 100.000000%
Train Epoch: 66/90 Train_Loss: 0.000382361700758338 Train_acc: 100.000000%
Train Epoch: 67/90 Train_Loss: 0.000287122413283214 Train_acc: 100.000000%
Train Epoch: 68/90 Train_Loss: 0.0006023799069225788 Train_acc: 100.000000%
Train Epoch: 69/90 Train_Loss: 0.00021455128444358706 Train_acc: 100.000000%
Train Epoch: 70/90 Train_Loss: 0.000745304103475064 Train_acc: 100.000000%
Train Epoch: 71/90 Train_Loss: 0.0002438421652186662 Train_acc: 100.000000%
Train Epoch: 72/90 Train_Loss: 0.0009173100115731359 Train_acc: 100.000000%
Train Epoch: 73/90 Train_Loss: 0.00048040799447335303 Train_acc: 100.000000%
Train Epoch: 74/90 Train_Loss: 0.0005091021303087473 Train_acc: 100.000000%
Train Epoch: 75/90 Train_Loss: 0.0005705688381567597 Train_acc: 100.000000%
Train Epoch: 7

Train Epoch: 79/90 Train_Loss: 0.16412800550460815 Train_acc: 94.687500%
Train Epoch: 80/90 Train_Loss: 0.131216362118721 Train_acc: 94.187500%
Train Epoch: 81/90 Train_Loss: 0.06427796930074692 Train_acc: 94.875000%
Train Epoch: 82/90 Train_Loss: 0.18283472955226898 Train_acc: 94.625000%
Train Epoch: 83/90 Train_Loss: 0.01719370111823082 Train_acc: 94.375000%
Train Epoch: 84/90 Train_Loss: 0.12354684621095657 Train_acc: 94.375000%
Train Epoch: 85/90 Train_Loss: 0.19327610731124878 Train_acc: 94.750000%
Train Epoch: 86/90 Train_Loss: 0.07718340307474136 Train_acc: 94.250000%
Train Epoch: 87/90 Train_Loss: 0.15971586108207703 Train_acc: 94.625000%
Train Epoch: 88/90 Train_Loss: 0.1194227933883667 Train_acc: 94.625000%
Train Epoch: 89/90 Train_Loss: 0.14489102363586426 Train_acc: 94.750000%
Train Epoch: 90/90 Train_Loss: 0.16315394639968872 Train_acc: 94.562500%
Test accuracy of the model: 75.000000%
Test accuracy of the model: 64.166667%
Test accuracy of the model: 58.750000%
Training T

Train Epoch: 1/90 Train_Loss: 0.0017248288495466113 Train_acc: 100.000000%
Train Epoch: 2/90 Train_Loss: 0.001553598791360855 Train_acc: 100.000000%
Train Epoch: 3/90 Train_Loss: 0.0011237190337851644 Train_acc: 100.000000%
Train Epoch: 4/90 Train_Loss: 0.0008566720644012094 Train_acc: 100.000000%
Train Epoch: 5/90 Train_Loss: 0.0011224676854908466 Train_acc: 100.000000%
Train Epoch: 6/90 Train_Loss: 0.0037708370946347713 Train_acc: 100.000000%
Train Epoch: 7/90 Train_Loss: 0.0009018955170176923 Train_acc: 100.000000%
Train Epoch: 8/90 Train_Loss: 0.0006278412183746696 Train_acc: 100.000000%
Train Epoch: 9/90 Train_Loss: 0.0007374489796347916 Train_acc: 100.000000%
Train Epoch: 10/90 Train_Loss: 0.0008893405320122838 Train_acc: 100.000000%
Train Epoch: 11/90 Train_Loss: 0.0005390534061007202 Train_acc: 100.000000%
Train Epoch: 12/90 Train_Loss: 0.0008196125854738057 Train_acc: 100.000000%
Train Epoch: 13/90 Train_Loss: 0.0007163506234064698 Train_acc: 100.000000%
Train Epoch: 14/90 Tra

In [5]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()